In [ ]:
!pip freeze | grep scikit-learn

In [ ]:
!python -V

In [ ]:
import pickle
import pandas as pd
import numpy as np
import sklearn
print(sklearn.__version__)

In [ ]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [7]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [8]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

### Q1

In [ ]:
np.std(y_pred)

### Q2

In [21]:
year = 2023
month = 3
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df['prediction'] = y_pred
df_result = df[['ride_id', 'prediction']]

In [ ]:
df

In [24]:
output_file = 'df_result'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

### Q3

In [ ]:
# jupyter nbconvert --to script your_notebook_name.ipynb